# Analysis Tools Examples

**Description:** Introduction to generating science performance diagnostic plots and metrics with the [analysis_tools](https://github.com/lsst/analysis_tools) package using a small test dataset from HSC.

**Contact authors:** Nate Lust, Keith Bechtol

**Last verified to run:** 2023-05-22

**LSST Science Piplines version:** Weekly 2023_20

**Container size:** medium

**Targeted learning level:** intermediate

**Skills:** 
* Generate a science performance diagnostic plot and corresponding metric values interactively in a notebook and as part of a pipeline (simple pipeline executor). 
* Adjust the configuration used to produce these diagnostics. 
* Retrieve persisted plots and metrics with the Bulter. 
* Reconstitute input data products that were used to create plots and metrics for further investigation.

## Preliminaries

In [ ]:
# Basic imports
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

from pprint import pprint

### Getting set up at USDF

The USDF is hosted on the S3DF cluster at SLAC. This notebook has been verified to run on the S3DF cluster.

See USDF documentation at
* https://developer.lsst.io/usdf/lsst-login.html
* https://developer.lsst.io/usdf/onboarding.html

### Processing rc2_subset

[rc2_subset](https://github.com/lsst-dm/rc2_subset) is a small dataset with just enough Hyper Suprime-Cam (HSC) exposures to compute a set of meaningful science performance metrics.

The LSST Science Pipelines [Getting Started tutorial](https://pipelines.lsst.io/#getting-started) provides a guided tour of data processing using rc2_subset as an example.

For convenience, there is a shell script `process_rc2_subset.sh` in the same directory as this notebook that shows the commands to process rc2_subset on the USDF.

### Setting up the analysis_tools package

Check the version of the stack you are using

In [ ]:
!eups list -s | grep lsst_distrib
!eups list -s | grep LOCAL

The `analysis_tools` package was added to `lsst_distrib` in August 2022, and accordingly, if you have set up the LSST Stack version `w_2022_32` or later, then you should be able to import `analysis_tools` directly in the notebook.

In [ ]:
import lsst.analysis.tools
print(lsst.analysis.tools.__file__)

If you are doing development on the `analysis_tools` package and want to test in a notebook, follow the guidance [here](https://nb.lsst.io/science-pipelines/development-tutorial.html). Brief version below (for work on the RSP at USDF):

1. In the termal, clone the [analysis_tools](https://github.com/lsst/analysis_tools) repo and set up the package

```
source /opt/lsst/software/stack/loadLSST.bash
setup lsst_distrib

# Choose file location for your repo
cd ~/repos/
git clone https://github.com/lsst/analysis_tools.git
cd analysis_tools
setup -k -r .
scons
```

2. Add the following line to `~/notebooks/.user_setups`

```
setup -k -r ~/repos/analysis_tools
```

Your local version of `analysis_tools` should now be accessible in a notebook.

## Generating consistent metric values and visualizations

### Load data for testing

In [ ]:
from lsst.daf.butler import Butler

#repo = "/project/sandbox/bechtol/rc2_subset/SMALL_HSC"
repo = "/fs/ddn/sdf/group/rubin/sandbox/bechtol/rc2_subset/SMALL_HSC/"
collection = "u/bechtol/step3"

#repo = "~/repos_lsst/rc2_subset/SMALL_HSC"
#collection = "u/natelust/step3"

butler = Butler(repo, collections=[collection])
registry = butler.registry

In [ ]:
# Display the available dataset types
for d in sorted(registry.queryDatasetTypes()): print(d.name)

In [ ]:
# There should be one objectTable_tract catalog available
sorted(registry.queryDatasets("objectTable_tract"))

In [ ]:
dataId = {"tract": 9813, "instrument": "hsc"}
tableName = "objectTable_tract"
objectTable = butler.get(tableName, dataId=dataId)
objectTable

In [ ]:
# Show the column names
objectTable.columns.values

### Generate a metric and plot from a single tool

* Instantiate a butler, load some data
* Pass loaded data to an `AnalysisTool` to make metrics and plots

In this example, we compute PSF model size residuals relative to the observed PSF size.

In [ ]:
from lsst.analysis.tools.atools import ShapeSizeFractionalDiff

In [ ]:
metric = ShapeSizeFractionalMetric()

In [ ]:
action = ShapeSizeFractionalDiff()
# An important step to ensure that the "prep" stage returns masked
# vectors for all relevant keys from the table. This is normally
# done automatically in pipelines.
action.populatePrepFromProcess()
action.process.calculateActions.stars.highSNSelector.threshold = 1500

In [ ]:
# Additional setup for the metric
plotInfo = dataId.copy()
plotInfo.update({
    'bands': 'grizy',
    'run': 'rc2',
    'tableName': tableName,
})
skymap = butler.get('skyMap')

In [ ]:
# Returns a dict, the first item is the plot which is shown below
results = action(objectTable, band='i', skymap=skymap, plotInfo=plotInfo)

In [ ]:
# print metric values
from lsst.verify.measurement import Measurement

for key, value in results.items():
    print(value if isinstance(value, Measurement) else f'{key}: {type(value)}')

## How it works 

### Terminology

Data Types
* Scalar - Something that is number like (int, float, numpy.float32 etc.)
* Vector - Something that is ndarray like
* KeyedData - Anything that is indexed by a string that can return a Vector, or Scalar

Analysis Structures
* ConfigurableAction - generic interface for function like objects (actions) that have state which can be set during configuration
* AnalysisAction - A ConfigurableAction subclass that is specialized for actions that function in analysis contexts
* AnalysisTool - A top level "container" of multiple AnalysisActions which performs one type of analysis

Below we dive into the later two in more detail

### Using AnalysisActions

* These are the atomic bits of analysis_tools; They can be combined together to make more complex actions, or used as part of an AnalysisTool
* Show some examples of using configurable actions like standalone functions. This is intended to provide users with more intution about how configurable actions work.
* Examples with KeyedDataActions, VectorActions (including selectors), and ScalarActions
* Show examples of configuration

In [ ]:
from lsst.analysis.tools.actions.vector import CalcShapeSize, ConvertFluxToMag

Let's create an example AnalysisAction to compute the measured PSF size for a set of stars from an object catalog.

In [ ]:
sizeCalculator = CalcShapeSize()

In [ ]:
# Inspect the configuration of this object.
pprint(sizeCalculator.toDict())

In [ ]:
# Inspect the required input schema, notice that we will need to provide the band information
sizeCalculator.getInputSchema()

In [ ]:
size = sizeCalculator(objectTable, band='i')
print(size)

In [ ]:
# Another example, this time to convert fluxes to magnitudes
mag = ConvertFluxToMag(vectorKey='{band}_psfFlux')(objectTable, band='i')

# Notice that the line above is equiavalent to the following
mag_alternate = ConvertFluxToMag(vectorKey='i_psfFlux')(objectTable)

assert np.allclose(mag, mag_alternate, equal_nan=True)

In [ ]:
plt.figure()
plt.scatter(mag, size, s=1)
plt.xlim(17.5, 30.)
plt.ylim(0, 5)

Let's remake that simple plot now selecting only the stars

In [ ]:
from lsst.analysis.tools.actions.vector import StarSelector

In [ ]:
star_selection = StarSelector()(objectTable, band='i')

In [ ]:
plt.figure()
plt.scatter(mag[star_selection], size.values[star_selection], s=1)
plt.xlim(17.5, 30.)
plt.ylim(0, 5)

We can also chain together AnalysisActions, as in the following example that produces an equivalent plot. The `analysis_tools` package frequently uses this approach of chaining together AnalysisActions.

In [ ]:
from lsst.analysis.tools.actions.vector import DownselectVector
from lsst.analysis.tools.actions.keyedData import AddComputedVector

band = "i"
objectTable = AddComputedVector(action=CalcShapeSize(), keyName="size")(objectTable, band=band)

# Note type(objectTable) is now a python dictionary instead of a pandas table, but since both
# "quack" like KeyedData so they can be used interchangably

objectTable = AddComputedVector(
    action=ConvertFluxToMag(vectorKey='{band}_psfFlux'),
    keyName="mag"
)(objectTable, band=band)

size = DownselectVector(vectorKey="size", selector=StarSelector())(objectTable, band=band)
mag = DownselectVector(vectorKey="mag", selector=StarSelector())(objectTable, band=band)

plt.figure()
plt.scatter(mag, size, s=1)
plt.xlim(17.5, 30.)
plt.ylim(0, 5)

### Generic interface for data
Actions are not restricted to tables or products loaded from the butler, KeyedData could also be things like dictionaries of numpy arrays.

In [ ]:
import lsst.analysis.tools.actions
dir(lsst.analysis.tools.actions)

In [ ]:
from lsst.analysis.tools.actions.scalar import StdevAction

# create some KeyedData
data = {"randomData": np.random.normal(0, 3, 100)}

# initialize an action, setting it to use the key set above
action = StdevAction(vectorKey="randomData")

# Run the action and print the results
print(f"The standard deviation is {action(data)}")

### Three conceptual steps in an `AnalysisTool`: prep, process, produce

As mentioned AnalysisTools can be thought of as executable containers of AnalysisActions. There are three different AnalysisActions, referred to as stages, named prep, process, and produce.
* Prep - Responsible for any initial selection and filtering of data
* Process - This is where any transformations or derived data should be computed
* Produce - Generates final plot or metric objects

The following examples will:
* Walk through the three stages of running an analysis tool in sequential lines of code, passing the output of one step as input to the next step
* Examine intermediate results

In [ ]:
# make a new instance of the AnalysisPlot we used earlier
action = ShapeSizeFractionalDiff()

action.populatePrepFromProcess()

# Run the action by stepping through each stage
stage1 = action.prep(objectTable, band='i')
print(stage1.keys())
stage2 = action.process(stage1, band='i')
print(stage2.keys())
figure = action.produce(stage2, band='i', skymap=skymap, plotInfo=plotInfo)


## Workflow examples

### Running analysis_tools as part of a pipeline

* **All examples in this notebook should use the simple pipeline executor** (here is how you do it in a notebook)
* We have a PipelineTask for each data product. A task can run multiple AnalysisTools that each produce a set of plots or set of metrics and are subclasses of AnalysisPipelineTask.
* Discuss an example yaml pipeline file (load the yaml)
* Provide the command to run the pipeline
* Show how to configure the pipeline, e.g., turning on or off different metrics and plots or changing other parameters

## Pipeline
This is a copy of the coaddQualityCore pipeline in analysis_tools, reproduced here for reference
```
description: |
  Tier1 plots and metrics to assess coadd quality
tasks:
  analyzeObjectTableCore:
    class: lsst.analysis.tools.tasks.ObjectTableTractAnalysisTask
    config:
      connections.outputName: objectTableCore
      plots.shapeSizeFractionalDiffScatter: ShapeSizeFractionalDiffScatterPlot
      metrics.shapeSizeFractionalMetric: ShapeSizeFractionalMetric
      plots.e1DiffScatter: E1DiffScatterPlot
      metrics.e1DiffScatterMetric: E1DiffMetric
      plots.e2DiffScatter: E2DiffScatterPlot
      metrics.e2DiffScatterMetric: E2DiffMetric
      metrics.skyFluxStatisticMetric: SkyFluxStatisticMetric
      metrics.skyFluxStatisticMetric.applyContext: CoaddContext
      python: |
        from lsst.analysis.tools.analysisPlots import *
        from lsst.analysis.tools.analysisMetrics import *
        from lsst.analysis.tools.contexts import *
  catalogMatchTract:
    class: lsst.analysis.tools.tasks.catalogMatch.CatalogMatchTask
    config:
      bands: ['u', 'g', 'r', 'i', 'z', 'y']
  refCatObjectTract:
    class: lsst.analysis.tools.tasks.refCatObjectAnalysis.RefCatObjectAnalysisTask
    config:
      bands: ['u', 'g', 'r', 'i', 'z', 'y']
      ```

In [ ]:
from lsst.ctrl.mpexec import SimplePipelineExecutor
from lsst.pipe.base import Pipeline

# set up an output collection with your username
#outputCollection = "u/nate2/analysisToolsExample"
outputCollection = "u/bechtol/analysisToolsExample"

# this can be skipped if you already have a read writable butler setup (above is read only)
butlerRW = SimplePipelineExecutor.prep_butler(repo, inputs=[collection], output=outputCollection)

# load in the pipeline to run
pipeline = Pipeline.from_uri("$ANALYSIS_TOOLS_DIR/pipelines/coaddQualityCore.yaml")

# override a configuration within a certain AnalysisTool
configKey = "atools.shapeSizeFractionalDiff.prep.selectors.snSelector.threshold"
pipeline.addConfigOverride("analyzeObjectTableCore", configKey, 400)

bands = ['g', 'r', 'i', 'z']
pipeline.addConfigOverride("analyzeObjectTableCore", "bands", bands)
pipeline.addConfigOverride("catalogMatchTract", "bands", bands)
pipeline.addConfigOverride("refCatObjectTract", "bands", bands)

# restrict processing to the same dataId used above
whereString = "tract = 9813 AND skymap = 'hsc_rings_v1'"

# Prevent the executor from dumping plots into the notebook
backend_ =  mpl.get_backend() 
mpl.use("Agg")

executor = SimplePipelineExecutor.from_pipeline(pipeline, where=whereString, butler=butlerRW)
quanta = executor.run(True)

# Restore the ability for plots to be put into the notebook
mpl.use(backend_)

# If you only want to run one plot in a task in the pipeline do the following prior to execution
# pipeline.addConfigOverride("analyzeObjectTableCore", "plots", None)
# pipeline.addConfigOverride("analyzeObjectTableCore", "plots", ShapeSizeFractionalDiffScatterPlot)

### Inspect the results

In [ ]:
# refresh our read-only butler to see the changes made. (it's generally a
# good idea to work on read-only things)
butler.registry.refresh()

# see what datasets exist; there should now be objectTableCore_metrics
# for d in sorted(butler.registry.queryDatasetTypes()): print(d.name)

# get the metric that was written
refs = sorted(butler.registry.queryDatasets("objectTableCore_metrics", collections=outputCollection))
refs[0].dataId
objectTable_metrics = butler.get("objectTableCore_metrics", dataId=refs[0].dataId, collections=outputCollection)
pprint(objectTable_metrics)

### Reconstruct the inputs to an analysis_tool

Analysis(Tools/Actions) allow the exact state of and AnalysisTools to be saved into the butler when a pipeline is run. This allows a user to 'reconstruct' things as they were when the tools were executed. This aids in debugging and deep diving into the data.

Below is an example of reconstructing one of the tasks that was run in the Pipeline above.

In [ ]:
from lsst.analysis.tools.tasks.reconstructor import reconstructAnalysisTools

# Read in just one task
label = "analyzeObjectTableCore"
taskState, inputData = reconstructAnalysisTools(butler, collection=outputCollection, label=label, dataId=dataId, callback=None)
pprint(taskState)
pprint(inputData)

Notice that we have access to the object table used to produce the diagnostics.

In [ ]:
inputData["data"]

We can now reproduce diagnostic plots.

In [ ]:
# The following configuration won't be needed in the future
taskState.plots.shapeSizeFractionalDiffScatter.produce.addSummaryPlot = False

# Rerun one of the plots
taskState.plots.shapeSizeFractionalDiffScatter(
    inputData['data'],
    band='i',
    skymap=None,
    plotInfo=_StandinPlotInfo()
)

# change some configuration to see the differences
taskState.plots.shapeSizeFractionalDiffScatter.prep.selectors.snSelector.threshold = 50
taskState.plots.shapeSizeFractionalDiffScatter(
    inputData['data'],
    band='i',
    skymap=None,
    plotInfo=_StandinPlotInfo()
)

# This could be run in stages like the above example to investigate issues

## Make a custom analysis tool

Let's take a look at what at how a metric is implemented before creating a new AnalysisTool

In [ ]:
# This is copied from lsst.analysis.tools.analysisMetrics.limitingMagnitudeMetric

from lsst.analysis.tools.interfaces import AnalysisMetric

class FiveSigmaPointSourceDepthMetric(AnalysisMetric):
    """Calculate the five-sigma point source depth of a visit, based on the
    PSF flux and its reported error. By default the calculation selects
    objects between 4.75 < S/N < 5.25, but these limits are configurable.
    The flux type to use for selection is also configurable. Both the median
    and mean 5-sigma depths are returned.
    """

    parameterizedBand: bool = False

    def setDefaults(self):
        super().setDefaults()

        self.prep.selectors.starSelector = StarSelector()
        self.prep.selectors.starSelector.columnKey = "extendedness"

        self.prep.selectors.snSelector = SnSelector()
        self.prep.selectors.snSelector.fluxType = "psfFlux"
        # Select between 4.75 < SNR < 5.25 to get a decent sample:
        self.prep.selectors.snSelector.threshold = 4.75
        self.prep.selectors.snSelector.maxSN = 5.25

        self.process.buildActions.mags = ConvertFluxToMag(vectorKey="psfFlux")
        self.process.calculateActions.median5sigmaDepth = MedianAction(vectorKey="mags")
        self.process.calculateActions.mean5sigmaDepth = MeanAction(vectorKey="mags")

        self.produce.units = {
            "median5sigmaDepth": "mag",
            "mean5sigmaDepth": "mag",
        }


As discussed above, AnalysisTools (AnalysisMetric is a specialized subclass) are container classes for AnalysisActions.

To make deploying metrics and plots as easy as possible Analysis(Metric/Plots) contain default AnalysisActions that enable new AnalysisTools to be created by simply setting configuration.

The default prep action allow specifiying keys to load from input data, and applying selectors. If calling an AnalysisTool directly, the required keys can be set automatically.

The default process action itself has 3 stages to it,
* buildActions - actions which build derived data
* filterActions - If derived data needs to be filtered it can go here
* calculateActions - Any final calculations that may need done can be put here

These stages run sequentually, and any stage are allowed to have no actions set to run in them.

AnalysisMetrics have a default action which allow values produced in process to be mapped to lsst.verify.Measurements

The produce stage of AnalysisPlots do not have a default, as this is where the plot to produce is to be set.

In [ ]:
from lsst.analysis.tools.actions.scalar import MedianAction
from lsst.analysis.tools.actions.vector import SnSelector

class DemoMetric(AnalysisMetric):
    def setDefaults(self):
        super().setDefaults()
        
        # select on high signal to noise obejcts
        # add in a signal to noise selector
        self.prep.selectors.snSelector = SnSelector()
        
        # set what key the selector should use when deciding SNR
        self.prep.selectors.snSelector.fluxType = "psfFlux"
        
        # select what threshold value is desireable for the selector
        self.prep.selectors.snSelector.threshold = 100
        
        # the final name in the qualification is used as a key to insert
        # the calculation into KeyedData
        self.process.calculateActions.medianValueName = MedianAction(vectorKey="psfFlux")
        
        # tell the metic what the units are for the quantity
        self.produce.units = {"medianValueName": "Jy"}
        
        # Rename the quanity prior to producing the Metric
        # (useful for resuable workflows that set a name toward the end of computation)
        self.produce.newNames = {"medianValueName": "DemoMetric"}

In [ ]:
# make some fake data
fluxes = np.random.normal(100000, 100, 500)
data = {"psfFlux": fluxes, "psfFluxErr": np.sqrt(fluxes)}
metric = DemoMetric()(data)
metric